# ResNet50v2
Link: https://tykimos.github.io/warehouse/2018-3-19-ISS_Python_Deep_Learning_Keras_with_Blocks_jmj.pdf

API: https://keras.io/api/applications/

In [31]:
# import tensorflow
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [32]:
# %pip install scikit-image

In [33]:
# %conda list

In [1]:
import argparse
import sys

# Tensorflow ans tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from skimage.transform import resize

# keras libraries
from tensorflow.keras import models
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Input, Activation, UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


# resnet50
# from keras.applications import ResNet50
# from keras.applications.resnet50 import ResNet50




print(tf.__version__)

2.1.0


In [35]:
# keras.backend
# %conda install keras


In [36]:
# # keras memory link: https://nanodir.tistory.com/39

# import os
# import tensorflow as tf
# import keras.backend.tensorflow_backend as KK
 
 
# def get_session(gpu_fraction=0.18):
#     '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
 
#     num_threads = os.environ.get('OMP_NUM_THREADS')
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
 
#     if num_threads:
#         return tf.Session(config=tf.ConfigProto(
#             gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
#     else:
#         return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
 
 
# KK.set_session(get_session())



In [37]:
# # https://www.tensorflow.org/guide/gpu?hl=ko#gpu_%EB%A9%94%EB%AA%A8%EB%A6%AC_%EC%A0%9C%ED%95%9C%ED%95%98%EA%B8%B0
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*3.2)])
#   except RuntimeError as e:
#     # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
#     print(e)

In [38]:
(X_train_org, y_train_org), (X_test_org, y_test_org) = tf.keras.datasets.mnist.load_data()
print(y_train_org[0])

5


In [39]:
# X_train_224 = resize(X_train_org, (begin():end(),224,224))
print(type(X_train_org.shape))
num_train,x_org, y_org = X_train_org.shape
num_test,x_org, y_org = X_test_org.shape

X_train = X_train_org.astype(np.float32) / 255.0
X_test = X_test_org.astype(np.float32) / 255.0

# number of classes (label. 0~9: 10)
n_classes = 10

y_train = tf.keras.utils.to_categorical(y_train_org, n_classes) ### label (class vector(integer) => binary class matrix): 범주형 인코딩
y_test = tf.keras.utils.to_categorical(y_test_org, n_classes) 
print(y_train[0])
y_train = y_train.astype(np.int32) # label => 
y_test = y_test.astype(np.int32)
print(y_train[0])

<class 'tuple'>
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0 0 0 0 0 1 0 0 0 0]


In [40]:
# X_train_64 = np.zeros((num_train,64,64))
# X_train_64[0]= resize(X_train_org[0], (64,64))
# print(X_train_64.shape)
# # plt.show(X_train_64[0])
# print(type(X_train_org))
# print(type(X_train_64))
# plt.imshow(X_train_64[0], cmap='gray')
# plt.colorbar()

In [41]:
# num_train,x_org, y_org = X_train_org.shape
# X_train_224 = np.zeros((num_train,224, 224))
# X_train_224[0]= resize(X_train_org[0], (224,224))

# plt.imshow(X_train_224[0], cmap='gray')
# plt.colorbar()

In [42]:
X_train_rgb = np.stack((X_train,)*3, axis=-1)
X_test_rgb = np.stack((X_test,)*3, axis=-1)
print(type(X_train))
print(X_train.shape)
print("===")
print(type(X_train_rgb))
print(X_train_rgb.shape)
print("===")
print(y_train_org.shape)

<class 'numpy.ndarray'>
(60000, 28, 28)
===
<class 'numpy.ndarray'>
(60000, 28, 28, 3)
===
(60000,)


In [47]:
# model
# def_input = Input(shape=(28,28,1))
def_input = Input(shape=(28,28,3))
ratio = int(224/28)
upsampled = UpSampling2D((ratio, ratio))(def_input)
# model =  tf.keras.applications.ResNet50V2(input_tensor=upsampled, weights=None, classes=n_classes)
model =  tf.keras.applications.ResNet50V2(input_tensor=upsampled, weights=None, classes=n_classes)
# model.add(Dense(n_classes, activation='softmax'))

In [13]:
model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 3)]  0                                            
__________________________________________________________________________________________________
up_sampling2d (UpSampling2D)    (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           up_sampling2d[0][0]              
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
_________________________________________________________________________________________

In [14]:
# 3. 모델 학습과정 설정하기
# model.compile(loss='categorical_crossentropy', optimizer='adam',
#               metrics=['accuracy']) # 최적화 알고리즘 설정

model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [15]:
# 4. 모델 학습시키기
# model.fit_generator(
#  train_generator, # 훈련셋 지정
#  steps_per_epoch=200, # 총 훈련셋 수 / 배치 사이즈 (= 1000/50)
#  epochs=150) # 전체 훈련셋 학습 반복 횟수 지정
model.fit(X_train_rgb, y_train, epochs=5, batch_size=32)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 463s 8ms/sample - loss: 0.1755 - accuracy: 0.9463
Epoch 2/5
60000/60000 [==============================] - 413s 7ms/sample - loss: 0.0623 - accuracy: 0.9821
Epoch 3/5
60000/60000 [==============================] - 419s 7ms/sample - loss: 0.0451 - accuracy: 0.9866
Epoch 4/5
60000/60000 [==============================] - 444s 7ms/sample - loss: 0.0400 - accuracy: 0.9885
Epoch 5/5
60000/60000 [==============================] - 444s 7ms/sample - loss: 0.0306 - accuracy: 0.9909


In [16]:
# 5. 모델 평가하기
# score=model.evaluate_generator(test_generator, steps=3) 

In [17]:
# model evaluation
# link: https://tykimos.github.io/2017/06/10/Model_Save_Load/
loss_and_metrics = model.evaluate(X_test_rgb, y_test, batch_size=32)
print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

10000/10000 [==============================] - 23s 2ms/sample - loss: 0.0483 - accuracy: 0.9869

loss_and_metrics : [0.04830843538441841, 0.9869]


In [40]:
model_name = 'mnist_resnet50v2_model' # should update this value by each model

keras_name = 'save/keras/'
# keras2tf_name = 'save/keras-tf/'
# tf2tfl_name = 'save/tf-tfl/'
tfl_name = 'save/tflite/'

ext_keras = '.h5'
ext_tfl = '.tflite'
temp = keras_name+model_name+ext_keras
# print(keras_name+model_name)
print(temp)

save/keras/mnist_resnet50v2_model.h5


In [43]:
# save model
model.save(temp)

In [45]:
# load model
model_temp = load_model(temp)

In [47]:
model_temp = load_model(temp)

In [48]:
model_temp.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 3)]  0                                            
__________________________________________________________________________________________________
up_sampling2d (UpSampling2D)    (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           up_sampling2d[0][0]              
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
_________________________________________________________________________________________

In [ ]:
# %pip list

In [51]:
# # https://kaen2891.tistory.com/28
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1' # 초기화할 GPU number

# # out of memory

# # import tensorflow as tf

# with tf.Graph().as_default():
#   gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)

In [62]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [57]:
# %pip install numba

In [59]:
from numba import cuda

In [61]:
from keras import backend as K
K.clear_session()
